
## Overview

Esse notebook cria uma tabela permanente no DBFS com os dados de Cartão Corporativo do Governo Federal. Esta tabela é considerada a tabela 'fato', pois contém os valores das transações. A tabela também foi normalizada para compor o modelo 'estrela'. 
Este notebook responde às questões propostas no PDF do MVP2.

In [0]:
from datetime import datetime
from pyspark.sql.functions import col, date_format

In [0]:
# File location and type
file_location = "/FileStore/tables/202401_CPGF-1.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"
encode = 'latin1'

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", encode)  \
  .load(file_location)

display(df)

In [0]:
cols_to_drop = ["NOME ÓRGÃO SUPERIOR", "NOME ÓRGÃO", "NOME UNIDADE GESTORA", "NOME PORTADOR", "NOME FAVORECIDO"]
df = df.drop(*cols_to_drop)


In [0]:
df = df.withColumn("DATA TRANSAÇÃO", date_format(col("DATA TRANSAÇÃO"), "dd/MM/yyyy"))

In [0]:
df = df.to_pandas_on_spark()

/databricks/spark/python/pyspark/sql/dataframe.py:5178: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [0]:
# df["DATA TRANSAÇÃO"] = df["DATA TRANSAÇÃO"].apply(lambda x: datetime.strptime(str(x), "%d/%m/%Y") if x not null else str(x))

In [0]:
df["VALOR TRANSAÇÃO"] = df["VALOR TRANSAÇÃO"].apply(lambda x: float(str(x).replace(",", ".")))

In [0]:
df = df[(df["CNPJ OU CPF FAVORECIDO"] != -1) & (df["CNPJ OU CPF FAVORECIDO"] != -2)]

In [0]:
column_names = ["cod_org_sup", "cod_org", "unid_gestora", 
                "ano_extrato", "mes_extrato", "cpf_portador", 
                "cnpj_cpf_fav", "transaction", "dt_trans", "vlr_trans"]

df.columns = column_names

In [0]:
df[["cod_org_sup", "cod_org", "unid_gestora"]] = df[["cod_org_sup", "cod_org", "unid_gestora"]].astype(int)

In [0]:
df = df.to_spark()

In [0]:
%sql
DROP TABLE IF EXISTS fato_cpgf

In [0]:
# cpgf -> cartão de pagamentos do governo federal
permanent_table_name = "fato_cpgf"
# df.write.format('parquet').saveAsTable(permanent_table_name)
df.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
select * from fato_cpgf limit 5

cod_org_sup,cod_org,unid_gestora,ano_extrato,mes_extrato,cpf_portador,cnpj_cpf_fav,transaction,dt_trans,vlr_trans
63000,63000,110161,2024,01,***.562.861-**,05136706000107,COMPRA A/V - R$ - APRES,null,720.0
63000,63000,110161,2024,01,***.195.852-**,19878460000145,COMPRA A/V - R$ - APRES,null,1290.0
63000,63000,110161,2024,01,***.195.852-**,04932603000182,COMPRA A/V - R$ - APRES,null,8.0
63000,63000,110161,2024,01,***.195.852-**,43283811020266,COMPRA A/V - R$ - APRES,null,16.5
63000,63000,110161,2024,01,***.212.021-**,07708904000387,COMPRA A/V - R$ - APRES,null,513.6


a) TOP 10 ÓRGÃOS DA ADMINISTRAÇÃO FEDERAL QUE MAIS GASTARAM COM CARTÃO CORPORATIVO EM JAN/2024

In [0]:
%sql
SELECT
    fc.cod_org, ol.ORG_LOTACAO, 
    ROUND(SUM(fc.vlr_trans), 2) AS total_gasto
FROM
    fato_cpgf fc
INNER JOIN 
    org_lotacao ol 
ON
    ol.COD_ORG_LOTACAO = fc.cod_org
GROUP BY
    fc.cod_org, ol.ORG_LOTACAO
ORDER BY
    total_gasto DESC
LIMIT 10

cod_org,ORG_LOTACAO,total_gasto
20101,Presidência da República,2221202.95
25000,Ministério da Saúde,100584.24
30802,Departamento de Polícia Rodoviária Federal,67798.53
36201,Fundação Oswaldo Cruz,47141.75
26429,Instituto Federal de Goiás,46737.75
26443,Empresa Brasileira de Serviços Hospitalares,42848.09
32396,Agência Nacional de Mineração,38104.58
44207,Instituto Chico Mendes de Conservação da Biodiversidade,34163.0
26249,Universidade Federal Rural do Rio de Janeiro,29963.61
26263,Universidade Federal de Lavras - MG,28829.16


b) TOP 10 SERVIDORES DA ADMINISTRAÇÃO FEDERAL QUE MAIS GASTARAM COM CARTÃO CORPORATIVO EM JAN/2024

In [0]:
%sql
SELECT  
  cs.NOME, ug.UORG_LOTACAO, 
  round(sum(fc.vlr_trans), 2) AS total_gasto
FROM 
  fato_cpgf fc
INNER JOIN
  cadastro_servidores cs
ON 
  fc.cpf_portador = cs.CPF
INNER JOIN
  qualificacao_servidores qs
ON
  qs.Id_SERVIDOR_PORTAL = cs.id_SERVIDOR_PORTAL
INNER JOIN
  unid_org ug
ON
  qs.COD_UORG_LOTACAO = ug.COD_UORG_LOTACAO  
GROUP BY
  cs.NOME, ug.UORG_LOTACAO
ORDER BY
  total_gasto DESC
LIMIT 10

NOME,UORG_LOTACAO,total_gasto
REBECA MEDEIROS FONSECA,CENTRO DE REGISTRO E CONTROLE ACADEMICO,40233.66
NISVAL FERREIRA GUIMARAES,DIRECAO GERAL CAMPUS INHUMAS,38000.58
RICARDO YANEZ NOGUEIRA,INSTITUTO NAC DE CONTROL QUALID EM SAUDE,27917.96
CRISTIANE FRENSCH PEREIRA,INSTITUTO DE TECNOLOGIA EM IMUNOBIOLOGIC,27250.36
FERNANDO FRANCISCO DE SOUZA,DIRECAO GERAL CAMPUS INHUMAS,24808.42
MARCIA MARIA HENGEMUHLE,HOSPITAL UNIVERSITARIO DA UFSM,24260.94
LEONARDO LIMA BERGAMINI,GERENCIA DE MEIO AMBIENTE E GEOGRAFIA,21431.98
PAULO CESAR DO NASCIMENTO CORREA,COORDENACAO GERAL INFRAESTRUTURA CAMPI,21417.6
LUCIANA BANDEIRA DE SOUZA,DIRETORIA DE PLANEJAMENTO,20116.83
ANTENOR FONSECA DE OLIVEIRA FILHO,DIVISAO OPERACIONAL,20116.83


c) ÓRGÃOS DA ADMNISTRAÇÃO FEDERAL QUE GASTARAM MAIS DO QUE A MÉDIA DE GASTOS EM JAN/2024

In [0]:
%sql
WITH avgspending AS (
    SELECT
        ROUND(AVG(total_gasto), 2) AS avg_gasto
    FROM (
        SELECT
            fc.cod_org,
            ol.ORG_LOTACAO,
            SUM(fc.vlr_trans) AS total_gasto
        FROM
            fato_cpgf fc
        INNER JOIN 
            org_lotacao ol 
        ON
            fc.cod_org = ol.COD_ORG_LOTACAO
        GROUP BY
            fc.cod_org,
            ol.ORG_LOTACAO
  )
)

SELECT
    fc.cod_org,
    ol.ORG_LOTACAO,
    ROUND(SUM(fc.vlr_trans), 2) AS total_gasto
FROM
    fato_cpgf fc
INNER JOIN 
    org_lotacao ol 
ON
    fc.cod_org = ol.COD_ORG_LOTACAO
GROUP BY
    fc.cod_org,
    ol.ORG_LOTACAO
HAVING
    SUM(fc.vlr_trans) > (SELECT avg_gasto FROM avgspending)
ORDER BY
    total_gasto DESC

cod_org,ORG_LOTACAO,total_gasto
20101,Presidência da República,2221202.95
25000,Ministério da Saúde,100584.24
30802,Departamento de Polícia Rodoviária Federal,67798.53
36201,Fundação Oswaldo Cruz,47141.75
26429,Instituto Federal de Goiás,46737.75
26443,Empresa Brasileira de Serviços Hospitalares,42848.09


d) COMPRAS NO CARTÃO CORPORATIVO, EM JAN/2024, COM EMPRESA PUNIDA, QUE CONSTE NA TABELA CEIS, DA CGU

In [0]:

%sql
SELECT
  ce.cnpj_sanc, cde.razao_social, ce.n_processo, 
  ce.categ_sancao, ce.dt_in_san, cp.dt_trans, ce.dt_fin_san, 
  cp.vlr_trans
FROM 
  ceis ce
INNER JOIN
  fato_cpgf cp
ON
  ce.cnpj_sanc = cp.cnpj_cpf_fav
INNER JOIN 
  cadastro_empresas cde
ON 
  ce.cnpj_sanc = cde.cnpj

cnpj_sanc,razao_social,n_processo,categ_sancao,dt_in_san,dt_trans,dt_fin_san,vlr_trans
02458330000150,MULTISERV - COMERCIO E SERVICOS LTDA,0200160588615,Declaração de Inidoneidade com prazo determinado,14/02/2020,null,null,930.0
07358914000178,CEQUIMICA LTDA,019.5050.2019.0020074-73,Suspensão,23/04/2024,null,04/08/2024,2315.9
32602639000133,LUKATONER SUPPLIES LTDA,303/23,Impedimento/proibição de contratar com prazo determinado,09/02/2024,null,09/02/2025,218.0


In [0]:

%sql
SELECT
  ce.cnpj_sanc, cde.razao_social, ce.n_processo, 
  ce.categ_sancao, ce.dt_in_san, cp.dt_trans, ce.dt_fin_san, 
  cp.vlr_trans
FROM 
  ceis ce
INNER JOIN
  fato_cpgf cp
ON
  ce.cnpj_sanc = cp.cnpj_cpf_fav
INNER JOIN 
  cadastro_empresas cde
ON 
  ce.cnpj_sanc = cde.cnpj
WHERE
  date(cp.dt_trans) BETWEEN date(ce.dt_in_san) AND date(ce.dt_fin_san)

cnpj_sanc,razao_social,n_processo,categ_sancao,dt_in_san,dt_trans,dt_fin_san,vlr_trans
